In [2]:
!head -10 ml-1m/ratings.dat


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [9]:
import pandas as pd
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)

In [13]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
pd.read_csv?

In [15]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [18]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [25]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [28]:
print len(user_movie_pairs), len(set(user_movie_pairs))

1000209 1000209


In [37]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_array = csr_matrix((data, (row, col)))



In [48]:
ratings_array.shape

(3953, 6041)

In [41]:
np.sum(ratings_array != 0)

1000209

In [54]:
import numpy as np
#X = np.array([[1,1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=0)
W = model.fit_transform(ratings_array) 
H = model.components_



In [57]:
H.shape


(5, 6041)

In [53]:
model.n_components_

5

In [76]:
X = ratings_array

In [78]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

In [80]:
error/X.shape[0]/X.shape[1]

0.00012841350131919519